In [1]:
import torch
import numpy as np
from tqdm import tqdm

from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data.dataloader import DataLoader

from preprocessor import AgeRecognitionPreprocessor
from dataset import AgeRecognitionDataset
from models import vit_l_16_age_recognizer
from loss import AgeRecognitionLoss

In [2]:
lr = 1e-3

IMAGE_DIR = './Cleaned/'
TRAINING_PAIRINGS = './training_data.csv'
BATCH_SIZE = 1
EPOCHES = 1000
DEVICE = 'cuda'

In [3]:
model = vit_l_16_age_recognizer().to(DEVICE)
loss = AgeRecognitionLoss().to(DEVICE)
preprocessor = AgeRecognitionPreprocessor()
dataset = AgeRecognitionDataset(triplet_csv_path=TRAINING_PAIRINGS, image_dir=IMAGE_DIR, preprocessor=preprocessor, kfolds=5, device=DEVICE)

In [4]:
optimizer = Adam(list(model.parameters()) + list(loss.parameters()), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=EPOCHES, eta_min=1e-6)

In [5]:
for epoch in range(EPOCHES):
    for fold in range(dataset.kfolds):
        training_dataset, validation_dataset = dataset.kfold_cross_validation(fold)
        training_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        model.train()
        for batch in tqdm(training_dataloader):
            # Batch shape: (N, Anchor-Positive-Negative, C, H, W)
            print(model.forward_features(batch).shape)


  0%|          | 0/4985 [00:14<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 11.99 GiB total capacity; 10.73 GiB already allocated; 0 bytes free; 11.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF